In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

import pandas as pd
import numpy as np
import cv2
import shutil
import yaml
import warnings
warnings.filterwarnings("ignore")

from glob import glob
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [ ]:
data_path = '/content/drive/MyDrive/Colab Notebooks/ATL/합성데이터 기반 객체 탐지 AI 경진대회'

In [ ]:
if os.path.exists(data_path + "/data/yolo_v8"):
    shutil.rmtree(data_path + "/data/yolo_v8")

if not os.path.exists(data_path + "/data/yolo_v8/train"):
    os.makedirs(data_path + "/data/yolo_v8/train")

if not os.path.exists(data_path + "/data/yolo_v8/valid"):
    os.makedirs(data_path + "/data/yolo_v8/valid")

if not os.path.exists(data_path + "/data/yolo_v8/test"):
    os.makedirs(data_path + "/data/yolo_v8/test")

In [ ]:
def make_yolo_dataset(image_paths, txt_paths, type="train"):
    for image_path, txt_path in tqdm(zip(image_paths, txt_paths if not type == "test" else image_paths), total=len(image_paths)):
        source_image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        image_height, image_width, _ = source_image.shape

        target_image_path = f"{data_path}/data/yolo_v8/{type}/{os.path.basename(image_path)}"
        cv2.imwrite(target_image_path, source_image)

        if type == "test":
            continue

        with open(txt_path, "r") as reader:
            yolo_labels = []
            for line in reader.readlines():
                line = list(map(float, line.strip().split(" ")))
                class_name = int(line[0])
                x_min, y_min = float(min(line[5], line[7])), float(min(line[6], line[8]))
                x_max, y_max = float(max(line[1], line[3])), float(max(line[2], line[4]))
                x, y = float(((x_min + x_max) / 2) / image_width), float(((y_min + y_max) / 2) / image_height)
                w, h = abs(x_max - x_min) / image_width, abs(y_max - y_min) / image_height
                yolo_labels.append(f"{class_name} {x} {y} {w} {h}")

        target_label_txt = f"{data_path}/data/yolo_v8/{type}/{os.path.basename(txt_path)}"
        with open(target_label_txt, "w") as writer:
            for yolo_label in yolo_labels:
                writer.write(f"{yolo_label}\n")

In [ ]:
image_paths = sorted(glob(data_path + "/data/train/*.png"))
txt_paths = sorted(glob(data_path + "/data/train/*.txt"))

train_images_paths, valid_images_paths, train_txt_paths, valid_txt_paths = train_test_split(image_paths, txt_paths, test_size=0.1, random_state=SEED)

make_yolo_dataset(train_images_paths, train_txt_paths, "train")
make_yolo_dataset(valid_images_paths, valid_txt_paths, "valid")
make_yolo_dataset(sorted(glob(data_path + "/data/test/*.png")), None, "test")

100%|██████████| 3400/3400 [20:14<00:00,  2.80it/s]


In [ ]:
with open(data_path + "/data/classes.txt", "r") as reader:
    lines = reader.readlines()
    classes = [line.strip().split(",")[1] for line in lines]

yaml_data = {
              "names": classes,
              "nc": len(classes),
              "path": f"{data_path}/data/yolo_v8/",
              "train": "train",
              "val": "valid",
              "test": "test"
            }

with open(data_path + "/data/yolocustom.yaml", "w") as writer:
    yaml.dump(yaml_data, writer)

In [ ]:
with open(data_path + '/data/yolocustom.yaml', 'r') as writer:
    yolocustom_yaml = yaml.safe_load(writer)
    display(yolocustom_yaml)

{'names': ['chevrolet_malibu_sedan_2012_2016',
  'chevrolet_malibu_sedan_2017_2019',
  'chevrolet_spark_hatchback_2016_2021',
  'chevrolet_trailblazer_suv_2021_',
  'chevrolet_trax_suv_2017_2019',
  'genesis_g80_sedan_2016_2020',
  'genesis_g80_sedan_2021_',
  'genesis_gv80_suv_2020_',
  'hyundai_avante_sedan_2011_2015',
  'hyundai_avante_sedan_2020_',
  'hyundai_grandeur_sedan_2011_2016',
  'hyundai_grandstarex_van_2018_2020',
  'hyundai_ioniq_hatchback_2016_2019',
  'hyundai_sonata_sedan_2004_2009',
  'hyundai_sonata_sedan_2010_2014',
  'hyundai_sonata_sedan_2019_2020',
  'kia_carnival_van_2015_2020',
  'kia_carnival_van_2021_',
  'kia_k5_sedan_2010_2015',
  'kia_k5_sedan_2020_',
  'kia_k7_sedan_2016_2020',
  'kia_mohave_suv_2020_',
  'kia_morning_hatchback_2004_2010',
  'kia_morning_hatchback_2011_2016',
  'kia_ray_hatchback_2012_2017',
  'kia_sorrento_suv_2015_2019',
  'kia_sorrento_suv_2020_',
  'kia_soul_suv_2014_2018',
  'kia_sportage_suv_2016_2020',
  'kia_stonic_suv_2017_2019'